# 🎯 Valentina Facial LoRA Trainer - Google Colab Edition

Este notebook treina uma LoRA de alta qualidade para a identidade facial da Valentina usando FLUX.1-dev + NSFW checkpoint + Midjourney LoRA no Google Colab.

## Stack de Modelos:
- **Base**: FLUX.1-dev (via checkpoint NSFW)
- **Checkpoint**: `John6666/nsfw-master-flux-lora-merged-with-flux1-dev-fp16-v10-fp8-flux`
- **Style LoRA**: Midjourney LoRA (aplicada como base)
- **Output**: LoRA facial da Valentina para uso local com mflux

⚠️ **Importante**: Execute as células em ordem e aguarde a conclusão de cada etapa.

In [ ]:
# --- Configurações Principais ---
BASE_MODEL_ID = "John6666/nsfw-master-flux-lora-merged-with-flux1-dev-fp16-v10-fp8-flux" # Checkpoint NSFW (FLUX.1-dev mergeado)
MIDJOURNEY_LORA_FILENAME = "midjourney_LORA.safetensors" # Nome do arquivo da LoRA Midjourney
DATASET_ZIP_FILENAME = "valentina_dataset.zip" # Nome do arquivo zip do dataset

# --- Caminhos no Ambiente Colab ---
COLAB_MODELS_PATH = "/content/models"
COLAB_DATASET_PATH = "/content/dataset"
COLAB_OUTPUT_PATH = "/content/output_lora"
COLAB_LOGS_PATH = "/content/logs"

# --- Parâmetros de Treinamento da LoRA Facial ---
INSTANCE_PROMPT = "a photo of vltna woman" # Token único para Valentina
CLASS_PROMPT = "a photo of a woman" # Prompt de classe para regularização

# Configurações de resolução e qualidade
RESOLUTION = 1024
CENTER_CROP = True
RANDOM_FLIP = False # Evitar flip para manter consistência facial

# Batch sizes otimizados para T4/V100
TRAIN_BATCH_SIZE = 1
GRADIENT_ACCUMULATION_STEPS = 8 # Batch efetivo de 8

# Learning rates otimizados para LoRA facial
LEARNING_RATE = 8e-5 # Menor para preservar características faciais
UNET_LR = 8e-5
TEXT_ENCODER_LR = 5e-6 # Muito menor para text encoder

# Scheduler e warmup
LR_SCHEDULER = "cosine_with_restarts"
LR_WARMUP_STEPS = 100
LR_NUM_CYCLES = 1

# Steps de treinamento (calculado: ~100-150 steps por imagem)
MAX_TRAIN_STEPS = 1000 # Para 7 imagens = ~142 steps/imagem
SAVE_STEPS = 200
VALIDATION_EPOCHS = 5

# Arquitetura LoRA otimizada
LORA_RANK = 128 # Rank alto para melhor qualidade facial
LORA_ALPHA = 64 # Alpha = rank/2 para estabilidade
LORA_DROPOUT = 0.1

# Configurações de precisão e otimização
MIXED_PRECISION = "bf16" # Melhor qualidade se suportado
USE_8BIT_ADAM = True
ADAM_BETA1 = 0.9
ADAM_BETA2 = 0.999
ADAM_WEIGHT_DECAY = 0.01
ADAM_EPSILON = 1e-8
MAX_GRAD_NORM = 1.0

# Memory optimization
GRADIENT_CHECKPOINTING = True
ENABLE_XFORMERS = True
USE_CPU_OFFLOAD = False # Manter na GPU para velocidade

# Regularização e qualidade
PRIOR_LOSS_WEIGHT = 1.0
SNR_GAMMA = 5.0 # Para melhor qualidade com ruído

# Checkpointing
CHECKPOINTING_STEPS = 200
CHECKPOINTS_TOTAL_LIMIT = 3
RESUME_FROM_CHECKPOINT = None

# Seeds para reprodutibilidade
SEED = 42

# Configurações da Midjourney LoRA
USE_MIDJOURNEY_LORA = True
MIDJOURNEY_LORA_WEIGHT = 0.8 # Peso maior para estilo consistente

print("⚙️ Configurações otimizadas carregadas:")
print(f"📊 Steps totais: {MAX_TRAIN_STEPS}")
print(f"🎯 Batch efetivo: {TRAIN_BATCH_SIZE * GRADIENT_ACCUMULATION_STEPS}")
print(f"🧠 LoRA Rank: {LORA_RANK}")
print(f"🎨 Usar Midjourney LoRA: {USE_MIDJOURNEY_LORA}")

## Célula 2: Setup do Ambiente Colab (Dependências)

In [ ]:
# 🔧 Instalação Otimizada de Dependências
print("📦 Instalando dependências otimizadas para treinamento FLUX...")

# Instalar PyTorch com CUDA
!pip install -q --upgrade pip setuptools wheel
!pip install -q torch==2.1.0 torchvision==0.16.0 torchaudio==2.1.0 --index-url https://download.pytorch.org/whl/cu121

# Instalar diffusers e dependências principais
!pip install -q git+https://github.com/huggingface/diffusers.git
!pip install -q transformers==4.36.0
!pip install -q accelerate==0.25.0
!pip install -q peft==0.7.1
!pip install -q safetensors==0.4.1

# Dependências de otimização
!pip install -q xformers==0.0.22.post7 --index-url https://download.pytorch.org/whl/cu121
!pip install -q bitsandbytes==0.41.3
!pip install -q triton==2.1.0

# Utilitários
!pip install -q opencv-python-headless==4.8.1.78
!pip install -q pillow==10.1.0
!pip install -q imageio==2.31.6
!pip install -q ftfy==6.1.1

# Logging e monitoramento
!pip install -q tensorboard==2.15.1
!pip install -q wandb==0.16.1

# Hugging Face
!pip install -q huggingface_hub==0.19.4
!pip install -q datasets==2.14.7

print("✅ Todas as dependências instaladas com sucesso!")
print("🔥 Ambiente otimizado para treinamento de LoRA FLUX")

In [ ]:
# 🖥️ Verificação e Otimização da GPU
import torch
import subprocess

print("🔍 Verificando configuração da GPU...")
!nvidia-smi --query-gpu=name,memory.total,memory.free --format=csv,noheader,nounits

# Detectar capacidades da GPU
gpu_name = subprocess.check_output(["nvidia-smi", "--query-gpu=name", "--format=csv,noheader"]).decode().strip()
print(f"📱 GPU detectada: {gpu_name}")

# Ajustar configurações baseado na GPU
if "T4" in gpu_name:
    print("🔧 Otimizações para Tesla T4")
    MIXED_PRECISION = "fp16"  # T4 funciona melhor com fp16
    TRAIN_BATCH_SIZE = 1
    GRADIENT_ACCUMULATION_STEPS = 6
elif "V100" in gpu_name:
    print("🔧 Otimizações para V100")
    MIXED_PRECISION = "fp16"
    TRAIN_BATCH_SIZE = 1
    GRADIENT_ACCUMULATION_STEPS = 8
elif "A100" in gpu_name or "H100" in gpu_name:
    print("🔧 Otimizações para GPU high-end")
    MIXED_PRECISION = "bf16"  # Melhor qualidade
    TRAIN_BATCH_SIZE = 2
    GRADIENT_ACCUMULATION_STEPS = 4
else:
    print("🔧 Configurações padrão")
    MIXED_PRECISION = "fp16"

print(f"✅ Configurações ajustadas: {MIXED_PRECISION}, batch={TRAIN_BATCH_SIZE}")

# Limpar cache da GPU
torch.cuda.empty_cache()
print(f"🧹 Cache da GPU limpo. Memória livre: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f}GB")

print("Instalando dependências... Por favor, aguarde.")
!pip install -q diffusers transformers accelerate bitsandbytes safetensors peft xformers huggingface_hub torch torchvision torchaudio --upgrade

print("Dependências instaladas.")

In [ ]:
from huggingface_hub import notebook_login

print("Por favor, faça login na sua conta Hugging Face para baixar os modelos.")
notebook_login()

## Célula 4: Criação da Estrutura de Diretórios no Colab

In [ ]:
# 📁 Criação da Estrutura de Diretórios
import os
from pathlib import Path

# Criar todos os diretórios necessários
directories = [
    COLAB_MODELS_PATH,
    COLAB_DATASET_PATH,
    COLAB_OUTPUT_PATH,
    COLAB_LOGS_PATH,
    f"{COLAB_DATASET_PATH}/instance_images",
    f"{COLAB_DATASET_PATH}/class_images",
    f"{COLAB_OUTPUT_PATH}/checkpoints",
    f"{COLAB_OUTPUT_PATH}/samples"
]

for directory in directories:
    os.makedirs(directory, exist_ok=True)
    print(f"📂 Criado: {directory}")

print("\n✅ Estrutura de diretórios criada com sucesso!")

## Célula 5: Upload e Preparação dos Dados (Dataset e LoRA Midjourney)

In [ ]:
# 📤 Upload e Preparação dos Dados
from google.colab import files
import zipfile
import shutil
from PIL import Image
import os

print("📥 === UPLOAD DO DATASET ===")
print(f"Por favor, faça upload do arquivo: {DATASET_ZIP_FILENAME}")
uploaded_dataset = files.upload()

if DATASET_ZIP_FILENAME in uploaded_dataset:
    print(f"📦 Extraindo {DATASET_ZIP_FILENAME}...")
    with zipfile.ZipFile(DATASET_ZIP_FILENAME, 'r') as zip_ref:
        zip_ref.extractall(COLAB_DATASET_PATH)
    
    # Encontrar as imagens extraídas
    instance_images_path = f"{COLAB_DATASET_PATH}/instance_images"
    
    # Verificar estrutura do dataset
    extracted_files = []
    for root, dirs, files in os.walk(COLAB_DATASET_PATH):
        for file in files:
            if file.lower().endswith(('.png', '.jpg', '.jpeg')):
                extracted_files.append(os.path.join(root, file))
    
    print(f"🖼️ Encontradas {len(extracted_files)} imagens")
    
    # Mover imagens para pasta de instância
    for i, img_path in enumerate(extracted_files):
        new_path = f"{instance_images_path}/valentina_{i:03d}.png"
        
        # Converter e redimensionar se necessário
        with Image.open(img_path) as img:
            # Converter para RGB se necessário
            if img.mode != 'RGB':
                img = img.convert('RGB')
            
            # Redimensionar mantendo aspect ratio
            img.thumbnail((RESOLUTION, RESOLUTION), Image.Resampling.LANCZOS)
            
            # Criar imagem quadrada com padding
            new_img = Image.new('RGB', (RESOLUTION, RESOLUTION), (255, 255, 255))
            paste_x = (RESOLUTION - img.width) // 2
            paste_y = (RESOLUTION - img.height) // 2
            new_img.paste(img, (paste_x, paste_y))
            
            new_img.save(new_path, 'PNG', quality=95)
        
        print(f"✅ Processada: {os.path.basename(img_path)} -> {os.path.basename(new_path)}")
    
    print(f"\n📊 Dataset preparado: {len(extracted_files)} imagens de instância")
    !ls -la {instance_images_path}
else:
    print(f"❌ ERRO: {DATASET_ZIP_FILENAME} não encontrado!")
    raise FileNotFoundError("Dataset não foi enviado")

print("\n📥 === UPLOAD DA MIDJOURNEY LORA ===")
if USE_MIDJOURNEY_LORA:
    print(f"Por favor, faça upload do arquivo: {MIDJOURNEY_LORA_FILENAME}")
    uploaded_lora = files.upload()
    
    if MIDJOURNEY_LORA_FILENAME in uploaded_lora:
        midjourney_path = f"{COLAB_MODELS_PATH}/{MIDJOURNEY_LORA_FILENAME}"
        shutil.move(MIDJOURNEY_LORA_FILENAME, midjourney_path)
        print(f"✅ Midjourney LoRA salva em: {midjourney_path}")
        
        # Verificar tamanho do arquivo
        file_size = os.path.getsize(midjourney_path) / (1024*1024)
        print(f"📏 Tamanho: {file_size:.1f}MB")
    else:
        print("⚠️ Midjourney LoRA não enviada. Treinamento continuará sem ela.")
        USE_MIDJOURNEY_LORA = False
else:
    print("⏭️ Midjourney LoRA desabilitada nas configurações")

print("\n✅ Upload e preparação dos dados concluídos!")

## Célula 6: Preparação do Modelo Base para Treinamento da LoRA Facial
Carrega o checkpoint NSFW, aplica e funde a LoRA Midjourney (se habilitada e compatível), e salva este novo modelo como base para o treinamento da LoRA facial.

In [ ]:
import torch
from diffusers import FluxPipeline
import gc

print("🚀 Iniciando preparação do modelo base...")

# Configurar dtype baseado na GPU
if torch.cuda.is_available():
    if torch.cuda.get_device_capability()[0] >= 8:  # Ampere+
        model_dtype = torch.bfloat16
        print("💎 Usando bfloat16 (GPU Ampere+)")
    else:
        model_dtype = torch.float16
        print("⚡ Usando float16 (GPU older)")
else:
    model_dtype = torch.float32
    print("🐌 Usando float32 (CPU)")

try:
    print(f"📦 Carregando modelo base: {BASE_MODEL_ID}")
    
    # Carregar pipeline com configurações otimizadas
    pipeline = FluxPipeline.from_pretrained(
        BASE_MODEL_ID,
        torch_dtype=model_dtype,
        use_safetensors=True,
        variant="fp16" if model_dtype != torch.float32 else None
    )
    
    print("🔧 Configurando pipeline para treinamento...")
    pipeline.to("cuda")
    pipeline.enable_model_cpu_offload()  # Otimização de memória
    
    if USE_MIDJOURNEY_LORA and os.path.exists(f"{COLAB_MODELS_PATH}/{MIDJOURNEY_LORA_FILENAME}"):
        print("🎨 Aplicando Midjourney LoRA...")
        
        try:
            # Carregar LoRA Midjourney
            pipeline.load_lora_weights(
                f"{COLAB_MODELS_PATH}/{MIDJOURNEY_LORA_FILENAME}",
                adapter_name="midjourney"
            )
            
            # Configurar peso da LoRA
            pipeline.set_adapters(["midjourney"], adapter_weights=[MIDJOURNEY_LORA_WEIGHT])
            
            print(f"✅ Midjourney LoRA aplicada com peso {MIDJOURNEY_LORA_WEIGHT}")
            
            # Salvar modelo intermediário
            intermediate_path = f"{COLAB_MODELS_PATH}/base_with_midjourney"
            print(f"💾 Salvando modelo intermediário em: {intermediate_path}")
            
            pipeline.save_pretrained(
                intermediate_path,
                safe_serialization=True,
                variant="fp16" if model_dtype != torch.float32 else None
            )
            
            model_path = intermediate_path
            print("🎯 Modelo intermediário salvo com sucesso!")
            
        except Exception as e:
            print(f"⚠️ Erro ao aplicar Midjourney LoRA: {e}")
            print("📝 Continuando com modelo base original...")
            model_path = BASE_MODEL_ID
            USE_MIDJOURNEY_LORA = False
    else:
        print("📋 Usando modelo base original (sem Midjourney LoRA)")
        model_path = BASE_MODEL_ID
    
    print(f"🏁 Modelo final preparado: {model_path}")
    
except Exception as e:
    print(f"❌ Erro crítico na preparação do modelo: {e}")
    raise

finally:
    # Limpeza de memória
    if 'pipeline' in locals():
        del pipeline
    gc.collect()
    torch.cuda.empty_cache()
    print("🧹 Memória limpa")

print("\n✅ Preparação do modelo concluída!")

## Célula 7: Treinamento da LoRA Facial

In [ ]:
print("Baixando script de treinamento Dreambooth LoRA para FLUX...")
!wget https://raw.githubusercontent.com/huggingface/diffusers/main/examples/dreambooth/train_dreambooth_lora_flux.py -O {COLAB_TRAINING_SCRIPT_PATH}

print("Construindo comando de treinamento...")
training_command = f"

accelerate launch {COLAB_TRAINING_SCRIPT_PATH} \
  --pretrained_model_name_or_path='{model_to_train_on}' \
  --instance_data_dir='{COLAB_ACTUAL_DATASET_IMAGES_PATH}' \
  --output_dir='{COLAB_OUTPUT_PATH}' \
  --instance_prompt='{INSTANCE_PROMPT}' \
  --resolution={RESOLUTION} \
  --train_batch_size={TRAIN_BATCH_SIZE} \
  --gradient_accumulation_steps={GRADIENT_ACCUMULATION_STEPS} \
  --learning_rate={LEARNING_RATE} \
  --lr_scheduler='{LR_SCHEDULER}' \
  --lr_warmup_steps={LR_WARMUP_STEPS} \
  --max_train_steps={MAX_TRAIN_STEPS} \
  --lora_rank={LORA_RANK_FACIAL} \
  --seed={SEED} \
  --mixed_precision='{MIXED_PRECISION}' \
  --checkpointing_steps={CHECKPOINTING_STEPS} \
  --checkpoints_total_limit={CHECKPOINTS_TOTAL_LIMIT} \
  --validation_prompt='A photo of {INSTANCE_PROMPT} in a vibrant city at night' \
  --validation_epochs=10 \
  --report_to='tensorboard'

,
,
if GRADIENT_CHECKPOINTING:
    training_command += " \
  --gradient_checkpointing"
if ENABLE_XFORMERS:
    training_command += " \
  --enable_xformers_memory_efficient_attention"

print("Comando de Treinamento:")
print(training_command)

print("
Iniciando treinamento... Isso pode levar um tempo considerável.")
# Executa o comando no shell
!{training_command}

# 🚂 Treinamento de LoRA de Alta Qualidade
import subprocess
import os

print("🔥 Iniciando treinamento da LoRA facial...")

# Baixar script de treinamento mais recente
training_script_url = "https://raw.githubusercontent.com/huggingface/diffusers/main/examples/dreambooth/train_dreambooth_lora_flux.py"
training_script_path = f"{COLAB_LOGS_PATH}/train_dreambooth_lora_flux.py"

print("📥 Baixando script de treinamento otimizado...")
!wget -q {training_script_url} -O {training_script_path}

# Verificar se o dataset está correto
instance_images_path = f"{COLAB_DATASET_PATH}/instance_images"
num_images = len([f for f in os.listdir(instance_images_path) if f.lower().endswith(('.png', '.jpg', '.jpeg'))])
print(f"📊 Treinando com {num_images} imagens de instância")

# Calcular steps otimizados
steps_per_image = 150  # Otimizado para qualidade facial
calculated_steps = num_images * steps_per_image
MAX_TRAIN_STEPS = min(calculated_steps, 1200)  # Máximo de 1200 steps

print(f"🎯 Steps calculados: {MAX_TRAIN_STEPS} ({steps_per_image} por imagem)")

# Construir comando de treinamento otimizado
training_command = f"""
accelerate launch {training_script_path} \
  --pretrained_model_name_or_path="{model_path}" \
  --instance_data_dir="{instance_images_path}" \
  --output_dir="{COLAB_OUTPUT_PATH}" \
  --instance_prompt="{INSTANCE_PROMPT}" \
  --resolution={RESOLUTION} \
  --train_batch_size={TRAIN_BATCH_SIZE} \
  --gradient_accumulation_steps={GRADIENT_ACCUMULATION_STEPS} \
  --learning_rate={LEARNING_RATE} \
  --lr_scheduler="{LR_SCHEDULER}" \
  --lr_warmup_steps={LR_WARMUP_STEPS} \
  --lr_num_cycles={LR_NUM_CYCLES} \
  --max_train_steps={MAX_TRAIN_STEPS} \
  --checkpointing_steps={CHECKPOINTING_STEPS} \
  --checkpoints_total_limit={CHECKPOINTS_TOTAL_LIMIT} \
  --seed={SEED} \
  --mixed_precision="{MIXED_PRECISION}" \
  --prior_loss_weight={PRIOR_LOSS_WEIGHT} \
  --snr_gamma={SNR_GAMMA} \
  --rank={LORA_RANK} \
  --alpha={LORA_ALPHA} \
  --target_modules="to_k" "to_q" "to_v" "to_out.0" \
  --validation_prompt="{INSTANCE_PROMPT}, professional portrait, high quality, detailed" \
  --validation_prompt="{INSTANCE_PROMPT}, smiling, natural lighting" \
  --validation_prompt="{INSTANCE_PROMPT}, elegant pose, studio lighting" \
  --num_validation_images=2 \
  --validation_epochs={VALIDATION_EPOCHS} \
  --logging_dir="{COLAB_LOGS_PATH}" \
  --report_to="tensorboard" \
  --push_to_hub=False"""

# Adicionar flags condicionais
if GRADIENT_CHECKPOINTING:
    training_command += " \\
  --gradient_checkpointing"

if USE_8BIT_ADAM:
    training_command += " \\
  --use_8bit_adam"

if ENABLE_XFORMERS:
    training_command += " \\
  --enable_xformers_memory_efficient_attention"

if not RANDOM_FLIP:
    training_command += " \\
  --no_hflip"

if CENTER_CROP:
    training_command += " \\
  --center_crop"

print("🔧 Comando de treinamento configurado:")
print("=" * 50)
print(training_command)
print("=" * 50)

print("\n🚀 INICIANDO TREINAMENTO... (isso pode levar 30-60 minutos)")
print("💡 Acompanhe o progresso nas saídas abaixo")

# Executar treinamento
try:
    result = subprocess.run(training_command, shell=True, capture_output=False, text=True)
    if result.returncode == 0:
        print("\n🎉 TREINAMENTO CONCLUÍDO COM SUCESSO!")
    else:
        print(f"\n⚠️ Treinamento finalizado com código: {result.returncode}")
except Exception as e:
    print(f"\n❌ Erro durante o treinamento: {e}")
    raise

print("\n📊 Verificando resultados...")
!ls -la {COLAB_OUTPUT_PATH}/
print("\n📈 Logs do Tensorboard disponíveis em:", COLAB_LOGS_PATH)

## Célula 8: Resultados (Compactar e Baixar)

In [ ]:
# 📦 Processamento e Download dos Resultados
import shutil
import zipfile
from pathlib import Path
from google.colab import files
import json

print("🔍 Analisando resultados do treinamento...")

# Verificar estrutura de saída
print(f"📁 Conteúdo da pasta de saída:")
!ls -la {COLAB_OUTPUT_PATH}/

# Encontrar a LoRA treinada
lora_files = []
for root, dirs, files in os.walk(COLAB_OUTPUT_PATH):
    for file in files:
        if file.endswith('.safetensors') and 'lora' in file.lower():
            lora_files.append(os.path.join(root, file))

print(f"\n🎯 Arquivos LoRA encontrados: {len(lora_files)}")
for lora_file in lora_files:
    size_mb = os.path.getsize(lora_file) / (1024*1024)
    print(f"  📄 {os.path.basename(lora_file)} ({size_mb:.1f}MB)")

# Identificar a LoRA final
final_lora_path = None
if lora_files:
    # Procurar pela LoRA final (geralmente a maior ou a mais recente)
    final_lora_path = max(lora_files, key=os.path.getmtime)
    print(f"\n✅ LoRA final identificada: {os.path.basename(final_lora_path)}")
else:
    # Procurar em checkpoints
    checkpoint_dirs = [d for d in os.listdir(COLAB_OUTPUT_PATH) if d.startswith('checkpoint-')]
    if checkpoint_dirs:
        latest_checkpoint = max(checkpoint_dirs, key=lambda x: int(x.split('-')[1]))
        checkpoint_path = f"{COLAB_OUTPUT_PATH}/{latest_checkpoint}"
        
        # Procurar LoRA no checkpoint
        checkpoint_lora = f"{checkpoint_path}/pytorch_lora_weights.safetensors"
        if os.path.exists(checkpoint_lora):
            final_lora_path = checkpoint_lora
            print(f"✅ LoRA encontrada no checkpoint: {latest_checkpoint}")

if final_lora_path:
    # Preparar pacote final
    final_package_dir = f"{COLAB_OUTPUT_PATH}/valentina_facial_lora_package"
    os.makedirs(final_package_dir, exist_ok=True)
    
    # Copiar LoRA final
    final_lora_name = "valentina_facial_lora.safetensors"
    shutil.copy2(final_lora_path, f"{final_package_dir}/{final_lora_name}")
    
    # Criar arquivo de configuração
    config = {
        "model_name": "Valentina Facial LoRA",
        "base_model": BASE_MODEL_ID,
        "midjourney_lora_used": USE_MIDJOURNEY_LORA,
        "midjourney_weight": MIDJOURNEY_LORA_WEIGHT if USE_MIDJOURNEY_LORA else None,
        "training_params": {
            "resolution": RESOLUTION,
            "max_train_steps": MAX_TRAIN_STEPS,
            "learning_rate": LEARNING_RATE,
            "lora_rank": LORA_RANK,
            "lora_alpha": LORA_ALPHA,
            "batch_size": TRAIN_BATCH_SIZE,
            "gradient_accumulation": GRADIENT_ACCUMULATION_STEPS
        },
        "usage_instructions": {
            "trigger_word": "vltna woman",
            "recommended_weight": "0.7-1.0",
            "compatible_with": "mflux, ComfyUI, A1111"
        }
    }
    
    with open(f"{final_package_dir}/config.json", 'w') as f:
        json.dump(config, f, indent=2)
    
    # Criar README
    readme_content = f"""# Valentina Facial LoRA

## Informações do Modelo
- **Modelo Base**: {BASE_MODEL_ID}
- **Tipo**: LoRA Facial para identidade da Valentina
- **Resolução de Treinamento**: {RESOLUTION}x{RESOLUTION}
- **Steps de Treinamento**: {MAX_TRAIN_STEPS}
- **Trigger Word**: `vltna woman`

## Como Usar

### No mflux (MacBook):
```bash
mflux-generate \
    --model "/path/to/base/model" \
    --lora "valentina_facial_lora.safetensors" \
    --lora-scale 0.8 \
    --prompt "a photo of vltna woman, [seu prompt aqui]"
```

### Prompts Recomendados:
- `a photo of vltna woman, professional portrait`
- `vltna woman, elegant dress, studio lighting`
- `a photo of vltna woman, natural smile, outdoor`

### Configurações Recomendadas:
- **Peso da LoRA**: 0.7 - 1.0
- **Steps**: 20-30
- **CFG Scale**: 7-9

## Notas
- Treinada com {num_images} imagens de referência
- Compatível com FLUX.1-dev
- Otimizada para geração facial de alta qualidade
"""
    
    with open(f"{final_package_dir}/README.md", 'w') as f:
        f.write(readme_content)
    
    # Copiar algumas imagens de validação se existirem
    validation_dir = f"{COLAB_OUTPUT_PATH}/validation_images"
    if os.path.exists(validation_dir):
        package_validation_dir = f"{final_package_dir}/sample_outputs"
        shutil.copytree(validation_dir, package_validation_dir, dirs_exist_ok=True)
        print("📸 Imagens de validação incluídas no pacote")
    
    # Criar arquivo ZIP
    zip_filename = "valentina_facial_lora_package.zip"
    with zipfile.ZipFile(zip_filename, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for root, dirs, files in os.walk(final_package_dir):
            for file in files:
                file_path = os.path.join(root, file)
                arc_name = os.path.relpath(file_path, final_package_dir)
                zipf.write(file_path, arc_name)
    
    print(f"\n📦 Pacote criado: {zip_filename}")
    print(f"📊 Tamanho do pacote: {os.path.getsize(zip_filename) / (1024*1024):.1f}MB")
    
    # Informações finais
    print("\n🎯 RESUMO DO TREINAMENTO:")
    print(f"✅ LoRA treinada com sucesso: {final_lora_name}")
    print(f"📈 Steps completados: {MAX_TRAIN_STEPS}")
    print(f"🎨 Midjourney LoRA usada: {'Sim' if USE_MIDJOURNEY_LORA else 'Não'}")
    print(f"🔧 Rank da LoRA: {LORA_RANK}")
    print(f"💪 Trigger word: {INSTANCE_PROMPT}")
    
    print("\n📥 Baixando pacote completo...")
    files.download(zip_filename)
    
    print("\n🎉 SUCESSO! LoRA da Valentina está pronta para uso no seu MacBook!")
    print("\n💡 Instruções:")
    print("1. Extraia o arquivo ZIP baixado")
    print("2. Coloque o arquivo .safetensors na pasta de LoRAs do mflux")
    print("3. Use o trigger word 'vltna woman' nos seus prompts")
    print("4. Comece com peso 0.8 e ajuste conforme necessário")
    
else:
    print("❌ ERRO: Nenhuma LoRA foi encontrada nos resultados!")
    print("🔍 Verificando logs para diagnóstico...")
    
    # Tentar encontrar logs de erro
    if os.path.exists(COLAB_LOGS_PATH):
        !find {COLAB_LOGS_PATH} -name "*.log" -o -name "events.out.tfevents.*" | head -5
    
    print("\n📋 Conteúdo completo da pasta de saída:")
    !find {COLAB_OUTPUT_PATH} -type f | head -20

## Célula 9: Limpeza (Opcional)
Descomente para remover arquivos grandes e economizar espaço no Colab se for continuar usando o runtime.

In [ ]:
# print("Limpando arquivos baixados e modelos intermediários...")
# !rm -rf {COLAB_INTERMEDIATE_MODEL_SAVE_PATH} # Remove o modelo base com Midjourney fundida
# !rm -rf {COLAB_MODELS_PATH}/{BASE_MODEL_ID.split('/')[-1]} # Remove o cache do modelo base original se baixado explicitamente
# !rm -f {DATASET_ZIP_FILENAME}
# !rm -f {COLAB_MIDJOURNEY_LORA_FULL_PATH} # Se não for mais necessário
# !rm -rf {COLAB_DATASET_PATH}/* # Limpa imagens extraídas
# print("Limpeza concluída (arquivos de modelo e dataset).")
# torch.cuda.empty_cache()